In [1]:
def kahan_sum(a):
    s = 0
    c = 0
    for i in range(len(a)):
        y = a[i] - c
        t = s + y
        c = (t - s) - y
        s = t
    return s

In [21]:
# also https://code.activestate.com/recipes/393090/
def frsum(iterable):
    return float(sum(imap(Fraction.from_float, iterable)))

In [20]:
#https://code.activestate.com/recipes/393090/
def msum(iterable):
    partials = []
    for x in iterable:
        i = 0
        for y in partials:
            if abs(x) < abs(y):
                x, y = y, x
            hi = x + y
            lo = y - (hi - x)
            if lo:
                partials[i] = lo
                i += 1
            x = hi
        partials[i:] = [x]
    return sum(partials, 0.0)
    

Testing the standard summation method for 3 tests:
1) will fractions give rounding error?
2) what about extremely large numbers?
3) what about extremely small numbers?
4) scientific notation?

In [22]:
'1) {0}'.format(sum([0.1]*10))            #should be 1

'1) 0.9999999999999999'

In [24]:
'2) {0}'.format(sum([3,10**30,-10**30]*20))            #should be 60

'2) 60'

In [26]:
'3) {0}'.format(sum([3,10**-15,-10**-15]*20))           #should be 60

'3) 60.0'

In [27]:
'4) {0}'.format(sum([3,10e20,-10e20]*20))              #should be 60

'4) 0.0'

The standard sum method cannot handle floats without rounding error, or scientific notation.
My first attempt at improving this was usign a kahan sum, testing with:
1) will fractions give rounding error?
2) will it woork with scientific notation

In [28]:
'1) {0}'.format(kahan_sum([0.1]*10))           #should be 1.0

'1) 1.0'

In [29]:
'2) {0}'.format(kahan_sum([3,10e20,-10e20]*20))           #should be 60

'2) 0.0'

The kahan sum made some improvements. But all it does is keep track of the decimals more closely. I did manage to find a sum that claims to be completely stable:
testing the msum method:
1) will fractions give round error?
2) scientific notation?

In [30]:
'1) {0}'.format(msum([0.1]*10))         #should be 1.0

'1) 1.0'

In [31]:
'2) {0}'.format(msum([3,10e20,-10e20]*20))             #should be 60

'2) 60.0'

In [70]:
'Bonus) {0}'.format(msum([3,10e-10,-10e-10]*20))                #should be 60

'Bonus) 60.0'

In [94]:
import math
def pairwise_sum(a):
    n = len(a)
    if n <= 3:
        s = a[0]
        for i in range(1,n):
            s += a[i]
    else:
        m = math.floor(n/2)
        s = pairwise_sum(a[:m]) + pairwise_sum(a[m:])
    return s

In [98]:
pairwise_sum([0.1]*10)

1.0

In [99]:
pairwise_sum([3,10**30,-10**30]*20)

60

In [100]:
pairwise_sum([3,1e20,-1e20]*20)

0.0

Coclusion - Pairwise is good with decimals but cannot handle scientific notation